Term: YAI 2021 summer session

Team Member: Dongha Kim, Jeongeun Lee, Junho Lee, Suyeong Choi.

# LeNet-5 FashionMNIST Classification
Implement
* Set hyper parameters
* Call dataset from torchvision & make dataloader
* Build a LeNet-5
* Instantiate LeNet & define loss function and optimizer
* Training
* Test

In [1]:
import torch
import torchvision

from extensions.easyrun import runtime_info, dataset_info, Trainer  # type: ignore

# Set seed
torch.manual_seed(777)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Environment check
print(runtime_info())

<Runtime Information>
OS version: 		macOS-11.5.1-arm64-arm-64bit
Python version:		3.8.10 | packaged by conda-forge
Torch version:		1.8.0
Torch device:		cpu


## Set hyper parameters

In [2]:
num_epochs = 20
batch_size = 64
learning_rate = 1e-3

## Load and Preprocess Dataset
* Use Train Dataset with training and validating (5:1)
* Use Test Dataset with testing after learning

In [3]:
# Set transform for preprocessing

dataset_dir = 'data'
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))
])

# Load dataset

train_dataset = torchvision.datasets.FashionMNIST(
    root=dataset_dir, train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.FashionMNIST(
    root=dataset_dir, train=False, download=True, transform=transform)

train_dataset, val_dataset = torch.utils.data.random_split(  # type: ignore
    train_dataset, [50000, 10000])  # split - 5:1

# Print Information

print(dataset_info(train_dataset, val_dataset, test_dataset, loader=False))

<Dataset Information>
Train Dataset: 		50000
Validation Dataset: 	10000
Test Dataset: 		10000


In [4]:
# Data loader

train_loader = torch.utils.data.DataLoader(  # type: ignore
    train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

val_loader = torch.utils.data.DataLoader(  # type: ignore
    val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

test_loader = torch.utils.data.DataLoader(  # type: ignore
    test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# Print Information

print(dataset_info(train_loader, val_loader, test_loader, loader=True))

<DataLoader Information>
Train Batch: 		781
Validation Batch: 	156
Test Batch: 		156


## Build a LeNet-5

In [5]:
class LeNet5(torch.nn.Module):

    def __init__(self) -> None:
        super(LeNet5,self).__init__()
        self.feature_extractor: torch.nn.Module = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1)),
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
            torch.nn.Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1)),
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
            torch.nn.Conv2d(16, 120, kernel_size=(4, 4), stride=(1, 1)),
            torch.nn.Tanh(),
        )
        self.classifier: torch.nn.Module = torch.nn.Sequential(
            torch.nn.Linear(120, 84),
            torch.nn.Tanh(),
            torch.nn.Linear(84, 10),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.feature_extractor(x)
        x = x.view(-1, 120)
        x = self.classifier(x)
        return x


## Train, Validate and Test
Trainer Source:
* https://github.com/kdha0727/Easyrun-Pytorch/blob/master/easyrun.py

In [6]:
# Run

model = LeNet5()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

with Trainer(
        model, 'CrossEntropyLoss', optimizer, num_epochs,
        train_loader, val_loader, test_loader,
        verbose=True, timer=True, snapshot_dir='.',
) as trainer:
    trainer.to(device)
    trainer()


<Start Learning> 				Total 20 epochs

Epoch 1
[Train]	 Average loss: 0.63816, 		Total accuracy: 76.53% 
[Eval]	 Average loss: 0.45017, 		Total accuracy: 84.01%

Epoch 2
[Train]	 Average loss: 0.42638, 		Total accuracy: 84.39% 
[Eval]	 Average loss: 0.40362, 		Total accuracy: 85.49%

Epoch 3
[Train]	 Average loss: 0.37809, 		Total accuracy: 86.01% 
[Eval]	 Average loss: 0.38107, 		Total accuracy: 86.24%

Epoch 4
[Train]	 Average loss: 0.34346, 		Total accuracy: 87.39% 
[Eval]	 Average loss: 0.35472, 		Total accuracy: 87.11%

Epoch 5
[Train]	 Average loss: 0.32100, 		Total accuracy: 88.14% 
[Eval]	 Average loss: 0.34431, 		Total accuracy: 87.80%

Epoch 6
[Train]	 Average loss: 0.30079, 		Total accuracy: 88.83% 
[Eval]	 Average loss: 0.33630, 		Total accuracy: 87.99%

Epoch 7
[Train]	 Average loss: 0.28604, 		Total accuracy: 89.36% 
[Eval]	 Average loss: 0.33043, 		Total accuracy: 88.10%

Epoch 8
[Train]	 Average loss: 0.26914, 		Total accuracy: 90.04% 
[Eval]	 Average loss: 0.32304, 		T